In [2]:
from tensorflow import keras
max_features=100000
(train_x, train_y), (test_x, test_y) = keras.datasets.imdb.load_data(
    num_words=max_features
)
print(len(train_x), "Training sequences")
print(len(test_x), "Validation sequences")

17465344/17464789 [==============================] - 0s 0us/step
25000 Training sequences
25000 Validation sequences


In [3]:
print(train_y[0])
maxlen=100
train_x = keras.preprocessing.sequence.pad_sequences(train_x, maxlen=maxlen)
test_x = keras.preprocessing.sequence.pad_sequences(test_x, maxlen=maxlen)


1


In [1]:
!pip install MiniAttention==0.1

  Created wheel for MiniAttention: filename=MiniAttention-0.1-cp36-none-any.whl size=4199 sha256=ab4e5b1f6910c884dbac080d7fea01596a3294e029ba96239bfb84808d1abe37
  Stored in directory: /root/.cache/pip/wheels/f0/0e/5b/919e3e6aef2737499fd02a38de719c62eb177d334d7f71df48
Successfully built MiniAttention


In [8]:
import MiniAttention.MiniAttention as ma
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras import backend as k
from keras.layers import LSTM,Dense,Flatten,Bidirectional
from keras.activations import softmax,relu,elu,sigmoid
from keras.optimizers import Adagrad
from keras.initializers import glorot_uniform   
from keras.regularizers import l2
from keras.constraints import min_max_norm
from keras.layers import Embedding,Input
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Layer


def network(inp_shape,max_features,out_shape,train_x,train_y,test_x,test_y):
    #embedding_map=create_glove_embedding(train_x)
    #emb_mat,word_idx,emb_dim=embedding_preprocess(embedding_map,max_features,inp_shape,train_x,train_y,test_x,test_y)
    inp=Input(shape=(inp_shape,))
    #freeze emb weights
    #z=tf.keras.layers.Embedding(word_idx, emb_dim, weights=[emb_mat], trainable=False)(inp)
    z=Embedding(max_features,256)(inp)
    
    z=ma.MiniAttentionBlock(keras.initializers.he_uniform,None,None,keras.regularizers.L2(l2=0.02),None,None,None,None,None)(z)
    
    z=tf.keras.layers.Bidirectional(LSTM(128,recurrent_activation="relu",return_sequences=True))(z)
    z=tf.keras.layers.Bidirectional(LSTM(64,recurrent_activation="relu",return_sequences=True))(z)
    z=ma.MiniAttentionBlock(keras.initializers.he_uniform,None,None,keras.regularizers.L2(l2=0.02),None,None,None,None,None)(z)
    z=keras.layers.Dense(64,activation="relu")(z)
    z=keras.layers.Dense(64,activation="relu")(z)
    z=keras.layers.Dense(1,activation="sigmoid")(z)
    model=keras.models.Model(inputs=inp,outputs=z)
    model.compile(loss="binary_crossentropy",metrics=['accuracy'],optimizer=keras.optimizers.Adagrad(learning_rate=1e-3))
    model.summary()
    return model
max_features=100000
emb_size=300
inp_shape=72
out_shape=1
maxlen=100
model=network(inp_shape,max_features,out_shape,train_x,train_y,test_x,test_y)  
model.fit(train_x,train_y,epochs=2,batch_size=16,verbose=1,validation_data=(test_x,test_y)) 



Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 72)]              0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 72, 256)           25600000  
_________________________________________________________________
mini_attention_block_1 (Mini (None, 72, 256)           0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 72, 256)           394240    
_________________________________________________________________
bidirectional_3 (Bidirection (None, 72, 128)           164352    
_________________________________________________________________
mini_attention_block_2 (Mini (None, 72, 128)           0         
_________________________________________________________________
dense (Dense)                (None, 72, 64)           